### RAG第一范式 - Naive RAG（朴素RAG）

<img src="../picture/img_17.png" width="50%">


1. 🔷【1】User + Query
- 这是起点，用户提出原始问题，和第一范式一样。

2. 🟦【2】Pre-Retrieval（查询前处理模块）
- ✅ 这是第二范式最显著的新增模块之一。

- 目的：让“用户原始问题”变得更适合检索（提高 Recall）

- 典型方法：

    - Query Rewriting（查询重写）：将口语化或不清晰的问题改写为更清晰明确的检索式。例如："最近AI发展怎么样" → "2023年人工智能技术发展趋势综述"

    - Query Expansion（查询扩展）：添加同义词、相关关键词，增加召回。

    - 语义结构化：将问题结构化为“主题 + 条件 + 目标”等。

- 用到的模型：T5、GPT、BART、BGE-Reranker。

3. 🟧【3】Documents + Indexing（文档与索引模块）
- 作用不变：将原始知识文档切割成 chunk，然后编码成向量，用于后续的检索。

- 区别点：

    - 第二范式中，索引过程通常不止一个维度（比如建多个索引库：语义向量库 + BM25 关键词库）。

    - 可以支持跨模态索引（如图文、图表）。

4. 🟨【4】Retrieval（检索模块）
- ✅ 这是第二范式中的“重点增强点”。

- 第一范式（Naive RAG）：

    - 仅使用一种向量检索模型（如 FAISS + SBERT）

- 第二范式的增强：

    - Hybrid Retrieval（混合检索）：同时使用 BM25（关键词）+ Dense Retrieval（语义）

    - Multi-Stage Retrieval（多阶段）：

        - Stage 1：用 Fast Dense Retriever 粗召回（如 DPR）

        - Stage 2：用 Cross Encoder 或 reranker 精排序（如 ColBERT）

- 目标：提升 召回质量 和 精确排序。

5. 🟩【5】Post-Retrieval（检索后处理模块）
- ✅ 第二范式核心中的“精炼”步骤。

- 主要任务：

    - Reranking：对候选文档重新排序，提高最相关 chunk 排在前面。

    - Context Compression：

        - 删除冗余内容，压缩上下文使其适配 LLM 输入长度限制。

        - 常见方法：TF-IDF 摘句、句向量去重、信息浓缩（如 Pegasus）。

6. ⚙️【6】Prompt 构造

- 构建 Prompt 提交给 LLM，与第一范式一致，但质量更高、上下文更相关、token 控制更智能。

7. 📩【7】Response（生成回复）
- 由 LLM（如 GPT-4）基于 Prompt 返回答案。

***总结：RAG 第二范式 = Naive RAG + 三大增强模块***

|  模块   | 增强点 | 实现方式 |
|:------:| :----: | :----: |
|  Pre-Retrieval   | 提升检索命中率 | Query Rewriting / Expansion |
|  Retrieval   | 提升召回质量 | DPR / Hybrid BM25+DPR / ColBERT |
|  Post-Retrieval   | 	提升上下文精度 | Rerank / 压缩 / 去冗余 |

#### 为什么第二范式比第一范式强？
- 因为它不仅找得更准（Pre+Post优化），还能处理更大、更复杂的上下文信息（多阶段筛选），非常适合实际部署和大规模知识库环境。

### 🧠 RAG 第二范式中的 Pre-Retrieval：Query Rewriting 模块详解
1. ✅ 一、为什么需要 Query Rewriting？
- 在 Naive RAG 中，原始用户查询直接用于向量检索。如果查询：

    - 太短、太泛化（如“最新政策”）

    - 表达不明确（如“它指的是谁？”）

    - 包含错别字或语法错误

    - 语义跨度大（如“请总结最近 AI 与法律交叉的一些趋势”）

- → 都可能导致 检索失败或检索结果质量低。

- 所以，*Query Rewriting（查询重写）*模块就是用于：

    - 将“用户自然语言问题”重写为“适合用于检索的查询表达”。

2. 🧩 二、Query Rewriting 的核心目标
|  目标   | 描述 |
|:------:| :----: |
|  语义明确   | 消除歧义，如“它”改为“ChatGPT” |
|  内容扩展	   | 补全背景，例如“最新进展” → “2024年4月人工智能生成内容的发展” |
|  检索友好	   | 适配向量检索器结构（比如使用实体化概念） |

3. 🔁 三、Query Rewriting 的核心方法
- 方法 1️⃣：基于模板的重写（Template-Based）
- 思路：预定义常见问句模板，替换关键词

- 例子：
    - 原始：猫有什么特点？
    - 重写：猫的物种行为、生理特征和习性有哪些？

- 优点：可控、精细

- 缺点：覆盖有限、不智能
- 方法 2️⃣：基于语言模型的生成式重写（LLM-Based）
- 思路：利用 T5、GPT 等生成模型，将原始查询翻译为“信息需求明确”的检索式

- 典型模型：

    - T5 + 监督训练：如 CANARD 数据集

    - Prompt-Based GPT 重写：即在 Prompt 中写明重写任务，如：

        - 你是搜索引擎助手，请将以下用户自然语言问题转换成更适合检索的句子：
        - 用户问题：猫的特点是什么？
        - 输出：
    - Multi-turn QA 处理：对于多轮对话，通过模型补全上下文。
- 例子（使用 GPT）：
    - 输入：它有哪些副作用？
    - 重写：抗VEGF蛋白药物 Earicimab 的副作用有哪些？
- 方法 3️⃣：Embedding 扩展（Query Expansion）
- 思路：通过同义词库或词向量找到语义相关词，拼接进查询。

- 工具：WordNet、BERT、BM25+Embedding联合检索

- 例子：
    - 原始：猫的特征
    - 扩展：猫的行为特征、生理特性、种类差异
4. 🧱 四、Query Rewriting 的结构逻辑
```
【输入】：用户原始查询
         ↓
【Query Rewriting 模块】
    - 上下文还原（如果是多轮）
    - 语义补全（补背景词、主语）
    - 表述标准化（将“它”变为明确名词）
         ↓
【输出】：结构清晰、语义丰富的“检索查询”
```
5. 🔄 五、与第一范式的核心区别
|  模块对比   | 第一范式（Naive RAG） | 第二范式（Advanced RAG） |
|:------:| :----: | :----: |
|  是否预处理 Query   | ❌ 直接使用原始用户输入 | ✅ 加入智能预处理 |
|  检索精度   | 易失败或偏离主题 | 显著提高 Recall 与精度 |
|  对模糊查询适应性   | 差 | 好 |
|  模型交互次数   | 少 | 多（可能用 GPT 或 T5 重写） |

6. 六、研究参考与典型论文
- 📘 "Learning to Rewrite Queries for Conversational Search" (SigIR 2020, CANARD)

- 📘 "Conversational Query Reformulation" (CQR) - 用于多轮QA中的 Query Rewriting

- 📘 "FiD + DPR" 系列模型（如 FiD-KD-Rerank）